# 00. 共通セットアップ

このノートブックは、演習全体で必要なライブラリのインストールや設定を行うためのものです。

**学習の前に、必ずこのノートブックを一度最後まで実行してください。**

## GPUの確認

Google ColabでGPUが正しく割り当てられているか確認します。
メニューの「ランタイム」→「ランタイムのタイプを変更」で、「ハードウェアアクセラレータ」が「T4 GPU」になっていることを確認してください。

In [ ]:
# 編集禁止セル
!nvidia-smi

## プロジェクトのセットアップ

演習用のファイルをGoogle Driveにコピーし、必要なライブラリをインストールします。

In [ ]:
# 編集禁止セル
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 編集禁止セル
# リポジトリをクローン（初回のみ）
import os

repo_path = '/content/llm_lab'
if not os.path.exists(repo_path):
    !git clone https://github.com/AkkeyLab/llm_lab.git
else:
    print('Repository already cloned.')

In [ ]:
# 編集禁止セル
# 作業ディレクトリへ移動
import os
os.chdir(repo_path)
!ls -F

## ライブラリのインストール

LLM、RAG、LoRA、Gradioなどを扱うためのライブラリをインストールします。
**注意：完了までに5〜10分程度かかります。**

In [ ]:
# 編集禁止セル
!pip install -q -U transformers==4.41.2
!pip install -q -U accelerate==0.30.1
!pip install -q -U bitsandbytes==0.43.1
!pip install -q -U sentence-transformers==2.7.0
!pip install -q -U faiss-gpu==1.7.2 # GPU版FAISS
!pip install -q -U peft==0.10.0
!pip install -q -U trl==0.8.6
!pip install -q -U datasets==2.19.0
!pip install -q -U gradio==4.31.5

インストールが完了したら、**一度ランタイムを再起動してください。**

メニューの「ランタイム」→「ランタイムを再起動」

## 共通モジュールのインポートと動作確認

ランタイム再起動後、ここから実行を再開します。

作成した `src` ディレクトリの共通モジュールをインポートし、正しく動作するか確認します。

In [ ]:
# 編集禁止セル
# Google Driveのマウントとディレクトリ移動（再起動後）
import os
from google.colab import drive
if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')

repo_path = '/content/llm_lab'
if os.path.exists(repo_path):
    os.chdir(repo_path)
else:
    print(f'Error: Directory not found at {repo_path}')

# srcディレクトリにパスを通す
import sys
sys.path.append(os.path.abspath('src'))

In [ ]:
# 編集禁止セル
# 共通モジュールのインポート
import torch
from src.common import load_llm, generate_text

print('PyTorch version:', torch.__version__)
print('GPU available:', torch.cuda.is_available())

### LLMのロードとテキスト生成テスト

`src/common.py` の `load_llm` 関数を使って、4bit量子化されたLLMをロードします。
初回はモデルのダウンロードに時間がかかります。

In [ ]:
# 編集禁止セル
# モデルのロード（4bit量子化）
# メモリを節約するため、このノートブックではテスト後にモデルを解放します
try:
    model, tokenizer = load_llm(use_4bit=True)
except Exception as e:
    print(f'モデルのロード中にエラーが発生しました: {e}')
    print('GPUメモリが不足している可能性があります。ランタイムを再起動して、このセルから再実行してください。')

In [ ]:
# 編集禁止セル
# テキスト生成のテスト
prompt = "日本で一番高い山はなんですか？"

if 'model' in locals() and 'tokenizer' in locals():
    generated_text = generate_text(model, tokenizer, prompt)

    print("--- プロンプト ---")
    print(prompt)
    print("\n--- 生成結果 ---")
    print(generated_text)
else:
    print("モデルがロードされていないため、テキスト生成をスキップしました。")

In [ ]:
# 編集禁止セル
# メモリ解放
import gc
if 'model' in locals(): del model
if 'tokenizer' in locals(): del tokenizer
gc.collect()
torch.cuda.empty_cache()
print("モデルを解放し、GPUキャッシュをクリアしました。")

## セットアップ完了

「生成結果」にテキストが表示されれば、セットアップは正常に完了です。

次の `01_gpt_baseline.ipynb` に進んでください。